# Scratch code for testing purposes and small experiments. 


In [114]:

import numpy as np
import importlib
from Graph import Graph
from helpers import *
import pandas as pd
import random
from tqdm import tqdm
from deterministic_attack import DeterministicAttack
from revisited_spectral import RevisitedSpectral
from erdos import SpectralAttack


%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Block matrix generation

In [115]:

# block = Graph.block_from_txt("datasets/block.txt")

# p_flip = 0.1

# for i in range(0, block.first_partition_size):
#     for j in range(block.first_partition_size+1, block.adj_matrix.shape[1]):
#         if block.adj_matrix[i, j] == 1 and np.random.rand() < p_flip:
#                 block.adj_matrix[i, j] = not block.adj_matrix[i, j]
#                 block.adj_matrix[j, i] = not block.adj_matrix[j, i]

#         elif block.adj_matrix[i, j] == 0 and np.random.rand() < p_flip:
#                 block.adj_matrix[i, j] = not block.adj_matrix[i, j]
#                 block.adj_matrix[j, i] = not block.adj_matrix[j, i]

# block.to_txt(f"datasets/block_{p_flip}.txt")




## Exploring the spectral approach

In [116]:
dataset = "cora"
G = Graph.from_txt(f"datasets/{dataset}.txt")
G1, G2 = G.split_dataset(common_prop=0.0, graph1_prop=0.0)
A = np.dot(G.adj_matrix, G.adj_matrix.T)

In [117]:
deter = DeterministicAttack(G1, A)
deter.run()
Gstar = deter.get_reconstructed_graph()
proba = RevisitedSpectral(Gstar, A)
n_nodes = Gstar.adj_matrix.shape[0]
proba.run(alpha=1, beta=len(G.edges()) / len(Gstar.edges()))
Gstar = proba.get_reconstructed_graph()
print("Errors after revisited:", np.sum(np.abs(Gstar.adj_matrix - G.adj_matrix)))
proba.sanity_check()
deter = DeterministicAttack(Gstar, A)
deter.run()
Gstar = deter.get_reconstructed_graph()
Gstar.fix_edges()

print("Errors :", np.sum(np.abs(Gstar.adj_matrix - G.adj_matrix)))


Triangle attack: 100%|██████████| 90/90 [00:00<00:00, 11356.42it/s]


+-------------------------------+-------+
|              Stat             | Value |
+-------------------------------+-------+
|   Number of impossible edges  |  3051 |
| Number of reconstructed edges |   90  |
|    Number of unknown edges    | 68869 |
+-------------------------------+-------+ 



Triangle attack: 100%|██████████| 90/90 [00:00<00:00, 12017.68it/s]


+-------------------------------+-------+
|              Stat             | Value |
+-------------------------------+-------+
|   Number of impossible edges  | 22548 |
| Number of reconstructed edges |   90  |
|    Number of unknown edges    | 49372 |
+-------------------------------+-------+ 



Triangle attack: 100%|██████████| 90/90 [00:00<00:00, 13495.19it/s]


...

+-------------------------------+-------+
|              Stat             | Value |
+-------------------------------+-------+
|   Number of impossible edges  | 70796 |
| Number of reconstructed edges |  907  |
|    Number of unknown edges    |  307  |
+-------------------------------+-------+ 



Triangle attack: 100%|██████████| 908/908 [00:00<00:00, 19604.09it/s]


+-------------------------------+-------+
|              Stat             | Value |
+-------------------------------+-------+
|   Number of impossible edges  | 71084 |
| Number of reconstructed edges |  908  |
|    Number of unknown edges    |   18  |
+-------------------------------+-------+ 



Triangle attack: 100%|██████████| 908/908 [00:00<00:00, 20126.35it/s]

+-------------------------------+-------+
|              Stat             | Value |
+-------------------------------+-------+
|   Number of impossible edges  | 71084 |
| Number of reconstructed edges |  908  |
|    Number of unknown edges    |   18  |
+-------------------------------+-------+ 

Errors : 12


In [118]:
A_prime = np.dot(Gstar.adj_matrix, Gstar.adj_matrix.T)
slots_of_error = np.argwhere(A != A_prime)
nodes_of_error = set()
for i, j in slots_of_error:
    nodes_of_error.add(i)
    nodes_of_error.add(j)

print(slots_of_error)


    # print(A[node, node])

# print(A[226, 22])

[[ 22  53]
 [ 22  54]
 [ 22 226]
 [ 22 227]
 [ 25 197]
 [ 25 250]
 [ 25 294]
 [ 25 295]
 [ 53  22]
 [ 53  53]
 [ 54  22]
 [ 54  54]
 [ 69 327]
 [ 69 328]
 [ 69 377]
 [ 69 378]
 [197  25]
 [197 197]
 [226  22]
 [226 226]
 [227  22]
 [227 227]
 [250  25]
 [250 250]
 [294  25]
 [294 294]
 [295  25]
 [295 295]
 [302 327]
 [302 328]
 [302 377]
 [302 378]
 [327  69]
 [327 302]
 [327 327]
 [328  69]
 [328 302]
 [328 328]
 [377  69]
 [377 302]
 [377 377]
 [378  69]
 [378 302]
 [378 378]]


In [119]:
for node in nodes_of_error:
    print(node, "A_prime = ", A_prime[node, node], "A = ", A[node, node])

print(np.argwhere(Gstar.adj_matrix[295] == 1))
print(np.argwhere(G.adj_matrix[295] == 1))

226 A_prime =  1 A =  2
227 A_prime =  1 A =  2
197 A_prime =  1 A =  2
294 A_prime =  1 A =  2
295 A_prime =  1 A =  2
69 A_prime =  18 A =  18
327 A_prime =  2 A =  3
328 A_prime =  2 A =  3
377 A_prime =  2 A =  3
378 A_prime =  2 A =  3
302 A_prime =  12 A =  12
53 A_prime =  1 A =  2
54 A_prime =  1 A =  2
22 A_prime =  11 A =  11
25 A_prime =  27 A =  27
250 A_prime =  1 A =  2
[[25]]
[[ 25]
 [294]]


# Experiments on graphs with the same G^2

In [120]:
import pandas as pd

g1 = pd.read_csv("datasets/josee.txt", sep="\t", header=None)
 

g2_matrix = np.array([[0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0],
                    [0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0],
                    [0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0],
                    [0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1],
                    [1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0],
                    [0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1],
                    [1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0],
                    [0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1],
                    [1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0],
                    [0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0],
                    [0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0],
                    [0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0],
                    ])


g1_matrix = Graph.from_txt("datasets/josee.txt").adj_matrix
# perm_1 = [0, 2, 4, 6, 8, 10, 1, 3, 5, 7, 9, 11]
# perm_2 = [1, 3, 5, 7, 9, 11, 0, 2, 4, 6, 8, 10]

g1_matrix_perm = g1_matrix[perm_1, :][:, perm_1]
g2_matrix_perm = g2_matrix[perm_2, :][:, perm_2]


G_square_1 = np.dot(g1_matrix, g1_matrix)
G_square_2 = np.dot(g2_matrix, g2_matrix)
# print(G_square_1)
# print(G_square_2)




rec = erdos(G_square, 0.5)
rec_perm1 = rec[perm_1, :][:, perm_1]

rec_pol = erdos(A, 0.5)

print("Errors G : ", np.linalg.norm(rec_pol - G.adj_matrix) ** 2)



NameError: name 'perm_1' is not defined